<a href="https://colab.research.google.com/github/AlifiyulAkyun/SparkMLLib/blob/main/spark2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Nama : Alifiyul Akyun
* No : 04
* Kelas : TI3B
* NIM : 2041720036

In [ ]:
# Connect Google Drive Untuk Ambil Data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=3efe1736c4766de2880e5a56d0ceb882dc27ebbcb86f116466374d261428f30d
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
# Import Library
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col


In [ ]:
spark = SparkSession.builder.appName("Movie Lens").getOrCreate()
sc = spark.sparkContext

In [ ]:
raw = sc.textFile("/content/drive/MyDrive/bigdata-master/spark/examples/mllib/ratings.dat")
mydata = [(2, 1), (3, 3), (4, 2)]
mydatardd = sc.parallelize(mydata).map(lambda x: Rating(0, x[0], x[1]))



In [ ]:
class Rating:
    def __init__(self, userId, movieId, rating):
        self.userId = userId
        self.movieId = movieId
        self.rating = rating


In [ ]:
# Parse String Menjadi Objek Rating
def parseRating(str):
    fields = str.split("::")
    assert len(fields) == 4
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]))

ratings = raw.map(parseRating)
ratings_df = ratings.toDF(["userId", "movieId", "rating"])

In [ ]:
# Convert the additional data to a DataFrame
mydata_df = mydatardd.toDF(["userId", "rating"])

In [ ]:
# Combine the ratings and additional data
totalratings_df = ratings_df.union(mydata_df)

In [ ]:
# Build the recommendation model using ALS
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(totalratings_df)

In [ ]:
# Generate product recommendations for user ID 1
products = model.recommendForUserSubset(spark.createDataFrame([(1,)]).toDF("userId"), 10)
# Load data from movies, join it, and display the names ordered by ratings

products.show()


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{1, 3741.0576}, ...|
+------+--------------------+

